# 1. Data collection

## 1.1 Get the list of animes

In [3]:
import numpy as np
import pandas as pd 
import requests
import time
import csv
import os
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup as bs
from datetime import datetime as dt
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer


In [2]:
def getUrls(start, stop): 

    urls = []

    for i in range(start, stop):
        url = 'https://myanimelist.net/topanime.php?limit='+str(i*50)
        r = requests.get(url)
        html_content = r.text
        soup = bs(html_content, 'lxml')
        links = soup.find_all('h3') 

        for anime in links[:-3]:
            if anime.find('a'):
                urls.append(anime.find('a')['href'])
    return urls

In [3]:
f = open("topAnime.txt", 'w', encoding="utf8")
f.write('\n'.join(getUrls(0,400)))
f.close

<function TextIOWrapper.close()>

## 1.2 Crawl animes

In [2]:
def saveHtml(page):
#saving the HTML file of page 'page' in the corresponding folder

    subfolder = "downloaded_Html/page_{}".format(page)
    os.makedirs(subfolder)

    f = open("topAnime.txt", 'r', encoding="utf8")
    lines = f.readlines()[(page-1)*50:(page)*50]
    lines = [line.rstrip() for line in lines]
    f.close

    i = 1+50*(page-1)
    for link in lines:
        html = requests.get(link)
        
        if html.status_code != 200:
            while(html.status_code != 200): 
                check = requests.get(link)
        
        file_name = '{}/{}.html'.format(subfolder, i)
        g = open(file_name, 'w', encoding="utf8")
        g.write(html.text)
        g.close
        i += 1

In [3]:
for i in range(379,400):
    saveHtml(i)

## 1.3 Parse downloaded pages

In [2]:
def getTitle(anime):
    return anime.strong.contents[0]

In [3]:
def getType(anime):
    return anime.find(text = 'Type:').find_next('a').contents[0]

In [4]:
def getNumEpis(anime):
    if anime.find(text = 'Episodes:').next_element.strip() != "Unknown":
        return int(anime.find(text = 'Episodes:').next_element.strip())
    else :
        return []

In [5]:
def getStart(anime):
    date = anime.find(text = 'Aired:').next_element.strip()
    
    if date != "Not available":
        if len(date.split(" to ")[0]) > 8:
            return dt.strptime(date.split(" to ")[0], '%b %d, %Y' )
        elif len(date.split(" to ")[0]) == 4:
            return dt.strptime(date.split(" to ")[0], '%Y' )
        elif len(date.split(" to ")[0]) == 8:
            return dt.strptime(date.split(" to ")[0], '%b %Y' )
        else:
            return pd.to_datetime(np.NaN, errors='coerce')
    else :
        return []

In [6]:
def getEnd(anime):
    date = anime.find(text = 'Aired:').next_element.strip()
    
    if date != "Not available":
        if len(date)>12 and len(date.split(" to ")[1]) > 8 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%b %d, %Y' )
        elif len(date)>12 and len(date.split(" to ")[1]) == 4 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%Y' )
        elif len(date)>12 and len(date.split(" to ")[1]) == 8 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%b %Y' )
        else:
            return pd.to_datetime(np.NaN, errors='coerce')
    else :
        return []        

In [7]:
def getNumMemb(anime):
    animeNumMembers = anime.find(text = 'Members:').next_element
    return int(animeNumMembers.replace('n', '').replace(',', '').strip())

In [8]:
def getScore(anime):
    if anime.find(text = 'Score:').find_next('span').contents[0] != 'N/A':
        animeScore = anime.find(text = 'Score:').find_next('span').contents
        return float(animeScore[0])
    else :
        return []            

In [9]:
def getUsers(anime):
    animeUsers = anime.find(text = 'Score:').find_next('span').find_next('span').contents
    if animeUsers[0] != 'Ranked:':
        return int(animeUsers[0])
    else :
        return []      

In [10]:
def getRank(anime):
    animeRank = anime.find(text = 'Ranked:').next_element
    if animeRank.replace('\n', '').strip() != 'N/A':
        return int(animeRank.replace('\n', '').replace('#', '').strip())
    else :
        return [] 

In [11]:
def getPopularity(anime):
    animePopularity = anime.find(text='Popularity:').next_element
    return int(animePopularity.replace("\n","").replace('#', '').strip())

In [12]:
def getDescription(anime):
    animeDescription = anime.find(text = 'Synopsis').find_next('p').text
    return animeDescription.replace("\n","")

In [13]:
def getRelated(anime):
    table = anime.find(text = 'Related Anime')
    animeRelated = []

    if(table != None):    
        table = table.find_next('table')
        table = table.find_all('a')

        for el in table:
            animeRelated.append(el.text)
                
    return animeRelated

In [14]:
def getCharact(anime):
    table = anime.find(text = 'Characters & Voice Actors').find_next('div')
    table = table.find_all('table')

    animeChar = []

    for el in table:
        people = el.find_all('h3')
        for person in people:
            animeChar.append(person.text)
        
    return animeChar

In [15]:
def getVoices(anime):
    table = anime.find(text = 'Characters & Voice Actors').find_next('div')
    table = table.find_all('h3')

    animeVoices = []

    for el in table:
        people = el.find_next('table')
        for person in people:
            animeVoices.append(person.find('a').text)
        
    return animeVoices

In [16]:
def getStaff(anime):
    
    table = anime.find_all(text = 'Staff')[1].find_next("div", {"class": "detail-characters-list clearfix"})
    animeStaff = []
    if(table != None):    
        table = table.find_all("table")
        for el in table:
            x = el.find_all("td")[1]
            person = [x.find("a").text, x.find("small").text]
            animeStaff.append(person)
    
    return animeStaff

In [17]:
pages = sorted(os.listdir('downloaded_Html')[1:], key = lambda page : int(page.split("_")[1]))

title = []
typ = []
numEpisode = []
start = []
end = []
numMembers = []
score = []
users = []
rank = []
popularity = []
synopsis = []
related = []
char = []
voices = []
staff = []

In [18]:
for page in pages:
    htmls = os.listdir('downloaded_Html/{}'.format(page))
    for i in range(1,1+len(htmls)):
        f = open("downloaded_Html/{}/{}.html".format(page,50*(int(page.split("_")[1])-1)+i), 'r', encoding="utf8")
        anime = bs(f, 'lxml')
        title.append(getTitle(anime))
        typ.append(getType(anime))
        numEpisode.append(getNumEpis(anime))
        start.append(getStart(anime))
        end.append(getEnd(anime))
        numMembers.append(getNumMemb(anime))
        score.append(getScore(anime))
        users.append(getUsers(anime))
        rank.append(getRank(anime))
        popularity.append(getPopularity(anime))
        synopsis.append(getDescription(anime))
        related.append(getRelated(anime))
        char.append(getCharact(anime))
        voices.append(getVoices(anime))
        staff.append(getStaff(anime))

In [21]:
col = ['animeTitle', 'animeType', 'animeNumEpisode','releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 
       'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff']

types = {'animeTitle' : 'object', 
         'animeType' : 'object', 
         'animeNumEpisode' : 'int64',
         'releaseDate' : 'datetime64', 
         'endDate' : 'datetime64', 
         'animeNumMembers' : 'int64', 
         'animeScore' : 'float64',
         'animeUsers' : 'int64', 
         'animeRank' : 'int64',
         'animePopularity' : 'int64',
         'animeDescription' : 'object',
         'animeRelated' : 'object',
         'animeCharacters' : 'object',
         'animeVoices' : 'object',
         'animeStaff' : 'object'}

df = pd.DataFrame(columns = col).astype(dtype = types) 

df.animeTitle = title 
df.animeType = typ 
df.animeNumEpisode = numEpisode 
df.releaseDate = start 
df.endDate = end 
df.animeNumMembers = numMembers 
df.animeScore = score 
df.animeUsers = users 
df.animeRank = rank 
df.animePopularity = popularity 
df.animeDescription = synopsis 
df.animeRelated = related 
df.animeCharacters = char 
df.animeVoices = voices 
df.animeStaff = staff 

In [22]:
df = df.to_csv("anime.tsv", sep='\t', index=False)

In [ ]:
def dataToTSV(page, df):
    
    subfolder = "tsv_files/page_{}".format(page)
    os.makedirs(subfolder)
    
    for i in range(len(df)):
        f = open("tsv_files/page_{}/anime_{}.tsv".format(page, i+50*(page-1)), 'w', encoding="utf8")
        tsv = csv.writer(f, delimiter = '\t')
        tsv.writerow([x for x in df.columns])
        tsv.writerow(x for x in df.iloc[i]) 

# 2. Search engine

## 2.1 Conjunctive query

In [4]:
df = pd.read_csv("anime.tsv", sep='\t')
df.head()

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff
0,Fullmetal Alchemist: Brotherhood,TV,64,2009-04-05 00:00:00,2010-07-04 00:00:00,2677887,9.16,1629051,1,3,After a horrific alchemy experiment goes wrong...,"['Fullmetal Alchemist', 'Fullmetal Alchemist',...","['Elric, Edward', 'Elric, Alphonse', 'Mustang,...","['Park, Romi', 'Kugimiya, Rie', 'Miki, Shinich...","[['Cook, Justin', 'Producer'], ['Yonai, Norito..."
1,Gintama°,TV,51,2015-04-08 00:00:00,2016-03-30 00:00:00,484270,9.09,169476,2,337,"Gintoki, Shinpachi, and Kagura return as the f...","['Gintama', 'Gintama Movie 2: Kanketsu-hen - Y...","['Sakata, Gintoki', 'Kagura', 'Shimura, Shinpa...","['Sugita, Tomokazu', 'Kugimiya, Rie', 'Sakaguc...","[['Fujita, Youichi', 'Director, Storyboard, Pl..."
2,Shingeki no Kyojin Season 3 Part 2,TV,10,2019-04-29 00:00:00,2019-07-01 00:00:00,1598256,9.09,1087519,3,33,Seeking to restore humanity's diminishing hope...,"['Shingeki no Kyojin', 'Shingeki no Kyojin Sea...","['Levi', 'Yeager, Eren', 'Ackerman, Mikasa', '...","['Kamiya, Hiroshi', 'Kaji, Yuki', 'Ishikawa, Y...","[['Yabuta, Shuuhei', 'Producer'], ['Wada, Jouj..."
3,Steins;Gate,TV,24,2011-04-06 00:00:00,2011-09-14 00:00:00,2092380,9.09,1115839,4,11,The self-proclaimed mad scientist Rintarou Oka...,"['Steins;Gate', 'ChäoS;HEAd', 'Robotics;Notes'...","['Okabe, Rintarou', 'Makise, Kurisu', 'Shiina,...","['Miyano, Mamoru', 'Imai, Asami', 'Hanazawa, K...","[['Iwasa, Gaku', 'Producer'], ['Yasuda, Takesh..."
4,Fruits Basket: The Final,TV,13,2021-04-06 00:00:00,2021-06-29 00:00:00,275939,9.07,113310,5,653,"Hundreds of years ago, the Chinese Zodiac spir...","['Fruits Basket', 'Fruits Basket 2nd Season', ...","['Souma, Kyou', 'Honda, Tooru', 'Souma, Yuki',...","['Uchida, Yuuma', 'Iwami, Manaka', 'Shimazaki,...","[['Ibata, Yoshihide', 'Director'], ['Aketagawa..."


In [5]:
synopsis = df.animeDescription

In [6]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to C:\Users\CASA-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
total = []
tokenizer = RegexpTokenizer(r"\w+")
ps = PorterStemmer()

for row in synopsis:
    tokens = tokenizer.tokenize(row.lower())
    for word in tokens:
        if word not in stop_words:
            total.append(ps.stem(word))
        
total = list(dict.fromkeys(total))

In [21]:
voc = {}
id = 0
for tok in total:
    voc[tok] = id
    id+=1

with open('vocabulary.csv', 'w', newline = '',encoding = 'utf8') as f:
    fieldnames = ['Word', 'term_id']
    w = csv.DictWriter(f, fieldnames = fieldnames,)
    w.writeheader()
    for key in voc:
        w.writerow({"Word": key, 'term_id': voc[key]})

In [22]:
vocabulary = pd.read_csv("vocabulary.csv")
vocabulary

,Word,term_id
0,horrif,0
1,alchemi,1
2,experi,2
3,goe,3
4,wrong,4
...,...,...
36564,nippl,36564
36565,kawabiraki,36565
36566,boatman,36566
36567,kisha,36567


In [62]:
def setDict(d,l):
    for i in l:
        d[i] = []
    return d

In [116]:
inverted_index = {}
inverted_index = setDict(inverted_index, vocabulary.Word)

for i in range(len(vocabulary.Word)):
    for key in vocabulary.Word:
        if synopsis[i].lower().find(str(key)) > 0:
            inverted_index[key].append(i)
    

In [120]:
with open('inverted_index.csv', 'w', newline = '',encoding = 'utf8') as f:
    fieldnames = ['term_id', 'document']
    w = csv.DictWriter(f, fieldnames = fieldnames,)
    w.writeheader()
    for key in inverted_index:
        w.writerow({"term_id": key, 'document': inverted_index[key]})

In [123]:
index = pd.read_csv("inverted_index.csv")
index

,term_id,document
10,left,[0]
11,catastroph,[0]
12,new,[0]
13,realiti,[]
14,ignor,[0]
15,alchem,[0]
16,principl,[0]
17,ban,[0]
18,human,"[0, 2]"
19,transmut,[0]
